In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fast-text-embeddings-without-subwords/wiki-news-300d-1M.vec/wiki-news-300d-1M.vec
/kaggle/input/fast-text-embeddings-without-subwords/crawl-300d-2M.vec/crawl-300d-2M.vec
/kaggle/input/googles-trained-word2vec-model-in-python/GoogleNews-vectors-negative300.bin.gz
/kaggle/input/googles-trained-word2vec-model-in-python/GoogleNews-vectors-negative300.bin
/kaggle/input/glove2word2vec/glove_w2v.txt
/kaggle/input/financial-sentiment-analysis/data.csv
/kaggle/input/glove6b50dtxt/glove.6B.50d.txt


**This notebook will give step by step approach to NLP problem from basic models to deep learning models** 

# Import files and libraries

In [2]:
pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 805 kB/s            
     |████████████████████████████████| 497.5 MB 21 kB/s              
     |████████████████████████████████| 2.1 MB 35.2 MB/s            
     |████████████████████████████████| 1.4 MB 35.3 MB/s            
     |████████████████████████████████| 462 kB 42.4 MB/s            
     |████████████████████████████████| 14.5 MB 28.5 MB/s            
     |████████████████████████████████| 5.8 MB 36.4 MB/s            
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.2
    Uninstalling tensorflow-2.6.2:
      Successfully uninstalled tensorflow-2.6.2
ERROR: pip's dependency res

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from gensim.models import KeyedVectors
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import catboost as cbt
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import pipeline
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.utils import np_utils
import tensorflow as tf
from keras.callbacks import EarlyStopping
import tensorflow_hub as hub
import tensorflow_text as text

In [4]:
df = pd.read_csv('/kaggle/input/financial-sentiment-analysis/data.csv')

df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [5]:
# mapping to the sentiment column 

dicto = {'positive': 1, 'neutral': 0 , 'negative': -1}

df.Sentiment = df.Sentiment.map(dicto)

In [6]:
ctv = CountVectorizer(tokenizer= word_tokenize, token_pattern=None)

ctv.fit_transform(df['Sentence'])


<5842x13215 sparse matrix of type '<class 'numpy.int64'>'
	with 111586 stored elements in Compressed Sparse Row format>

# Model-1 Logistic + BOW

In [7]:
#Baseline model let's start with a logistic regression model since it is the 
#fastest for high dimensional sparse data

from sklearn import linear_model
from sklearn import model_selection

df['kfold'] = -1

df = df.sample(frac=1).reset_index(drop= True)

y = df.Sentiment.values

# Intitiate kfold class from model_selection module

kf = model_selection.StratifiedKFold(n_splits = 5)

for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f

count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern=None)
# we go over the folds
for fold_ in range(5):
    train_df = df[df.kfold != fold_].reset_index(drop=True)
    test_df = df[df.kfold == fold_].reset_index(drop=True)
    count_vec.fit(train_df.Sentence)
    
    xtrain = count_vec.transform(train_df.Sentence)
    xtest = count_vec.transform(test_df.Sentence)
    
    model = linear_model.LogisticRegression(solver = 'liblinear')
    
    model.fit(xtrain,train_df.Sentiment)
    
    preds = model.predict(xtest)
    
    accuracy_precision = precision_score(test_df.Sentiment, preds, average='micro')
    accuracy_recall = recall_score(test_df.Sentiment, preds, average = 'micro')
    
    print('precision score:', accuracy_precision)
    print('recall score:', accuracy_recall)
    print("")
    
print(classification_report(test_df.Sentiment, preds, labels=[1, -1, 0]))

precision score: 0.6826347305389222
recall score: 0.6826347305389222

precision score: 0.6852010265183918
recall score: 0.6852010265183918

precision score: 0.6986301369863014
recall score: 0.6986301369863014

precision score: 0.6678082191780822
recall score: 0.6678082191780822

precision score: 0.6909246575342466
recall score: 0.6909246575342466

              precision    recall  f1-score   support

           1       0.77      0.73      0.75       370
          -1       0.30      0.22      0.26       172
           0       0.72      0.80      0.76       626

    accuracy                           0.69      1168
   macro avg       0.60      0.58      0.59      1168
weighted avg       0.68      0.69      0.68      1168



We can see that labels **1,0 have 70 percent** score but label **-1 score is pretty low**, hence we can conclude that we need to go on modelling untill these scores get better.

# Model 2 NaiveBayes + BOW

In [8]:
# Lets try with NaiveBayes model 

df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df['kfold'] = -1

kf = model_selection.StratifiedKFold(n_splits = 5)

for fold_ ,(t_,v_) in enumerate(kf.split(X= df, y=y)):
    df.loc[v_, 'kfold'] = fold_
    
for f in range(5):
    df_train = df[df.kfold != f].reset_index(drop=True)
    df_test = df[df.kfold == f].reset_index(drop=True)
    
    count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern=None)
    
    x_train = count_vec.fit_transform(df_train.Sentence)
    x_test = count_vec.transform(df_test.Sentence)
    
    model = MultinomialNB()
    
    model.fit(x_train, df_train.Sentiment)
    
    y_pred = model.predict(x_test)
    
    accuracy_precision = precision_score(df_test.Sentiment, y_pred, average = 'micro')
    
    #accuracy_recall = recall_score(df_test.Sentiment, y_pred)
    
    print('precision score:', accuracy_precision)
    #print('recall score:', accuracy_recall)
    print("")
    

print(classification_report(df_test.Sentiment, y_pred, labels=[1,-1,0]))    
    

precision score: 0.7108639863130881

precision score: 0.6629597946963216

precision score: 0.7071917808219178

precision score: 0.6943493150684932

precision score: 0.714041095890411

              precision    recall  f1-score   support

           1       0.77      0.67      0.71       370
          -1       0.46      0.33      0.38       172
           0       0.73      0.85      0.79       626

    accuracy                           0.71      1168
   macro avg       0.65      0.61      0.63      1168
weighted avg       0.70      0.71      0.70      1168



It is evident that the for **label -1 the the metrics values improved** so this model will be the new baseline model for this problem.

# Model-3 Naive Bayes with TFIDF

In [9]:
# Now lets try with TF-IDF vectorizer instead of bag of words to MultinomialNB().

df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df['kfold'] = -1

kf = model_selection.StratifiedKFold(n_splits = 5)

for f_,(t_,v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f_
    
for fold in range(5):
    df_train = df[df.kfold != fold].reset_index(drop = True)
    df_test = df[df.kfold == fold].reset_index(drop = True)
    
    tf_vec = TfidfVectorizer(tokenizer = word_tokenize, token_pattern=None)
    
    xtrain = tf_vec.fit_transform(df_train.Sentence)
    xtest = tf_vec.transform(df_test.Sentence)
    
    model = MultinomialNB()
    
    model.fit(xtrain, df_train.Sentiment)
    
    y_pred = model.predict(xtest)
    
    
    accuracy_precision = precision_score(df_test.Sentiment, y_pred, average = 'micro')

    print('precision score:', accuracy_precision)

    print("")
    

print(classification_report(df_test.Sentiment, y_pred, labels=[1,-1,0]))    
    
   

precision score: 0.6526946107784432

precision score: 0.6561163387510693

precision score: 0.6575342465753424

precision score: 0.6318493150684932

precision score: 0.6575342465753424

              precision    recall  f1-score   support

           1       0.71      0.41      0.52       370
          -1       1.00      0.03      0.06       172
           0       0.64      0.98      0.78       626

    accuracy                           0.66      1168
   macro avg       0.78      0.47      0.45      1168
weighted avg       0.72      0.66      0.59      1168



# Model-4 Naive Bayes with Ngrams

In [10]:
# Now to the baseline bag of word Naivebayes model lets apply Ngrams  and compare the results.

# Lets try with NaiveBayes model 

df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df['kfold'] = -1

kf = model_selection.StratifiedKFold(n_splits = 5)

for fold_ ,(t_,v_) in enumerate(kf.split(X= df, y=y)):
    df.loc[v_, 'kfold'] = fold_
    
for f in range(5):
    df_train = df[df.kfold != f].reset_index(drop=True)
    df_test = df[df.kfold == f].reset_index(drop=True)
    
    count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern=None, ngram_range = (1,3))
    
    x_train = count_vec.fit_transform(df_train.Sentence)
    x_test = count_vec.transform(df_test.Sentence)
    
    model = MultinomialNB()
    
    model.fit(x_train, df_train.Sentiment)
    
    y_pred = model.predict(x_test)
    
    accuracy_precision = precision_score(df_test.Sentiment, y_pred, average = 'micro')
    
    #accuracy_recall = recall_score(df_test.Sentiment, y_pred)
    
    print('precision score:', accuracy_precision)
    #print('recall score:', accuracy_recall)
    print("")
    

print(classification_report(df_test.Sentiment, y_pred, labels=[1,-1,0]))    

precision score: 0.6578272027373824

precision score: 0.6441402908468776

precision score: 0.6669520547945206

precision score: 0.660958904109589

precision score: 0.6549657534246576

              precision    recall  f1-score   support

           1       0.77      0.66      0.71       370
          -1       0.20      0.14      0.17       172
           0       0.68      0.80      0.73       626

    accuracy                           0.65      1168
   macro avg       0.55      0.53      0.54      1168
weighted avg       0.64      0.65      0.64      1168



So the **ngrams has even decreased the metrics** value.Hence it is evident that the baseline model is still the best one till now

# Text processing

I will **not remove stopwords** in this because it might change the context of the sentence.

e.g **"He is not a good person"** will be changed into **" 'He' , 'good', 'person'"** which is the complete opposite of the sentence.

In [11]:
# Now lets do some cleaning on the text data and apply it to baseline model and compare the accuracies.



def process_text(text):
    
    text = word_tokenize(text)
    text = [re.sub('[^A-Za-z]+', '', word) for word in text]
    text = [word.lower() for word in text if word.isalpha()]
    text = [WordNetLemmatizer().lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

In [12]:
text = 'He is not a good person'
token_text = word_tokenize(text)
[ word for word in token_text if word not in stopwords.words('english')]

['He', 'good', 'person']

In [13]:
df.Sentence = df.Sentence.apply(process_text)

# Model 5 cleaned text Naive Bayes BOW and 

In [14]:
#Now lets try this on our baseline MultinomialNB bagofwords model

df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df['kfold'] = -1

kf = model_selection.StratifiedKFold(n_splits = 5)

for fold_ ,(t_,v_) in enumerate(kf.split(X= df, y=y)):
    df.loc[v_, 'kfold'] = fold_
    
for f in range(5):
    df_train = df[df.kfold != f].reset_index(drop=True)
    df_test = df[df.kfold == f].reset_index(drop=True)
    
    count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern=None)
    
    x_train = count_vec.fit_transform(df_train.Sentence)
    x_test = count_vec.transform(df_test.Sentence)
    
    model = MultinomialNB()
    
    model.fit(x_train, df_train.Sentiment)
    
    y_pred = model.predict(x_test)
    
    accuracy_precision = precision_score(df_test.Sentiment, y_pred, average = 'micro')
    
    #accuracy_recall = recall_score(df_test.Sentiment, y_pred)
    
    print('precision score:', accuracy_precision)
    #print('recall score:', accuracy_recall)
    print("")
    

print(classification_report(df_test.Sentiment, y_pred, labels=[1,-1,0]))    

precision score: 0.7151411462788708

precision score: 0.7057313943541489

precision score: 0.714041095890411

precision score: 0.7071917808219178

precision score: 0.6926369863013698

              precision    recall  f1-score   support

           1       0.73      0.64      0.68       370
          -1       0.43      0.33      0.37       172
           0       0.72      0.83      0.77       626

    accuracy                           0.69      1168
   macro avg       0.63      0.60      0.61      1168
weighted avg       0.68      0.69      0.68      1168



Hurray!!! after some text cleaning **we beat our baseline model by small margin** hence the above model will be our new baseline model.

Now lets go into **word embeddings** 

In the above models each word token is converted into integer tokens by (BOW and TFIDF) now lets convert these **integer tokens into vectors.**

# Model 6 Fastext vector with Naive Bayes Baseline 

In [15]:
# In this model we will use fastText vectors and also convert each word vector in sentence vector.

#The code is taken from https://fasttext.cc/docs/en/english-vectors.html, this code splits each vector by 
# space and return for more info go through the above linl
 
import io

def sentence_to_vec(s, embedding_dict, tokenizer):
    
    words = tokenizer(s)
    
    embedding_list = []
    for w in words:
        
        if w in embedding_dict:
            embedding_list.append(embedding_dict[w])
            
    # if we dont have any vectors, then return zeros
    if len(embedding_list) == 0:
        return np.zeros(300)
    
    #convert list of embeddings into an array
    embedding_list = np.array(embedding_list)
    print
    #calculate sum over axis = 0 (This will convert the word vectorrs to sentence vectors)
    v = embedding_list.sum(axis=0)
    
    #return normalized vector
    return v

df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df.drop('kfold', axis=1, inplace = True)

kf = model_selection.StratifiedKFold(n_splits = 5)

#load embeddings into memory
print("Loading embeddings")
embeddings = KeyedVectors.load_word2vec_format('/kaggle/input/fast-text-embeddings-without-subwords/crawl-300d-2M.vec/crawl-300d-2M.vec')
    
#create sentence embeddings
print("creating sentence vectors")
vectors = []
for sentence in df.Sentence.values:
    vectors.append(         
       sentence_to_vec(s = sentence, embedding_dict= embeddings, tokenizer = word_tokenize)
    )
vectors = np.array(vectors)
    
for fold_,(train_, valid_) in enumerate(kf.split(X=df, y=y)):
        print("fold: ", fold_)
        
        xtrain = vectors[train_,:]
        ytrain = y[train_]
        
        xtest = vectors[valid_,:]
        ytest = y[valid_]
        
        scaler = MinMaxScaler()
        
        xscaled_train = scaler.fit_transform(xtrain)
        xscaled_test = scaler.transform(xtest)
        
        model = MultinomialNB()
    
        model.fit(xscaled_train,ytrain)
    
        y_pred = model.predict(xscaled_test)
    
        pres_score = precision_score(ytest, y_pred, average='micro')
    
        rec_score = recall_score(ytest, y_pred, average='micro')
    
        print('precision and recall scores:', pres_score, rec_score)
        print("")
    
print(classification_report(ytest, y_pred, labels=[1,-1,0]))

Loading embeddings
creating sentence vectors
fold:  0
precision and recall scores: 0.5372112917023096 0.5372112917023096

fold:  1
precision and recall scores: 0.5363558597091531 0.5363558597091531

fold:  2
precision and recall scores: 0.5368150684931506 0.5368150684931506

fold:  3
precision and recall scores: 0.5368150684931506 0.5368150684931506

fold:  4
precision and recall scores: 0.5368150684931506 0.5368150684931506

              precision    recall  f1-score   support

           1       1.00      0.00      0.01       370
          -1       0.00      0.00      0.00       172
           0       0.54      1.00      0.70       626

    accuracy                           0.54      1168
   macro avg       0.51      0.33      0.23      1168
weighted avg       0.60      0.54      0.38      1168



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Whoa! It is our worst performing model so far! What should we do now? Should we ask muppets to save us? No! we will keep on trying with other models and methods.Thats what data scientists do!**

# Model 7 - Glove vectors with Baseline model

In [16]:
def read_glove_vector(glove_vec):
    with open(glove_vec, 'r', encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            w_line = line.split()
            curr_word = w_line[0]
            word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
    return word_to_vec_map

In [17]:
class Word2VecVectorizer:
    def __init__(self, model):
        print("Loading in word vectors...")
        self.word_vectors = model
        print("Finished loading in word vectors")
    
    def fit(self, data):
        pass
    
    def transform(self, data):
        # determine the dimensionality of vectors
        v = self.word_vectors.get_vector('king')
        self.D = v.shape[0]

        X = np.zeros((len(data), self.D))
        n = 0
        emptycount = 0
    
        for sentence in data:
            tokens = sentence.split()
            vecs = []
            m = 0
            for word in tokens:
                try:
                    vec = self.word_vectors.get_vector(word)
                    vecs.append(vec)
                    m += 1
                except KeyError:
                    pass
            if len(vecs) > 0:
                vecs = np.array(vecs)
                X[n] = vecs.mean(axis=0)
            else:
                emptycount += 1
            n += 1
        print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
        return X


    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

In [18]:
from gensim.models import KeyedVectors

word2vec_output_file = '/kaggle/input/glove2word2vec/glove_w2v.txt'

model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary = False)

Xtrain,Xtest,ytrain,ytest = train_test_split(df.Sentence, df.Sentiment, test_size = 0.2, random_state = 42)

In [19]:
#we create a Vectorizer object that will help us to transform our reviews to vectors, a numerical representation. 
#Then we can use those vectors to feed our classifier.

vectorizer = Word2VecVectorizer(model)

X_train = vectorizer.fit_transform(Xtrain)
y_train = ytrain

X_test = vectorizer.transform(Xtest)
y_test = ytest

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 0 / 4673
Numer of samples with no words found: 0 / 1169


In [20]:
# Define the parameters to tune
parameters = { 
    'C': [1.0, 10],
    'gamma': [1, 'auto', 'scale']
}

model = GridSearchCV(SVC(kernel='rbf'), parameters, cv=5, n_jobs=-1).fit(X_train, y_train)

#model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred, labels=[1,-1,0]))

              precision    recall  f1-score   support

           1       0.74      0.67      0.70       359
          -1       0.41      0.20      0.27       181
           0       0.71      0.86      0.78       629

    accuracy                           0.70      1169
   macro avg       0.62      0.58      0.58      1169
weighted avg       0.68      0.70      0.68      1169



**Model 5 is still our best performing model, lets do some hyperparameter tuning before we move on to deep learning models**

# Model # Hyperparameter tuning best models

In [21]:
from sklearn.model_selection import GridSearchCV

count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern = None)

X_train = count_vec.fit_transform(df.Sentence)

parameters = {'alpha': [0.001,0.01,0.1,0.2,0.3,0.5,0.7,1,1.5,1.6,1.8,10,100]}

grid_search = MultinomialNB()

grid_search = GridSearchCV(grid_search, parameters, cv=5, scoring = 'f1_micro', n_jobs = -1)

grid_result = grid_search.fit(X_train, df.Sentiment)

print('Best params: ', grid_result.best_params_)
print('Best score: ', grid_result.best_score_)



Best params:  {'alpha': 1.6}
Best score:  0.7113974594841629


In [22]:
# Lets also check svm hyperparameters using pipelines

df = df.sample(frac=1).reset_index(drop=True)



count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern = None)

X_train = count_vec.fit_transform(df.Sentence)

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, cv=5, scoring = 'f1_micro', n_jobs = -1)
 
# fitting the model for grid search
grid.fit(X_train, df.Sentiment)

print('Best params: ', grid.best_params_)
print('Best score: ', grid.best_score_)



Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  1.7min


[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.536, total=   8.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.536, total=   8.2s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.536, total=   8.4s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.564, total=   7.6s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.558, total=   7.9s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] ....... C=0.1, gamma=0.01, kernel=rbf, score=0.536, total=   6.2s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] ....... C=0.1, gamma=0.01, kernel=rbf, score=0.536, total=   6.2s
[CV] C

[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  7.4min finished


Best params:  {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Best score:  0.6906901461265337


Lets use grid search for hyperparameterization.Eventhough its computationally expensive it may perform very good on our model and dataset lets see!

# Model 8 Best basline model+hyperparameters

In [23]:
#Now lets try this on our baseline MultinomialNB bagofwords model

df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df['kfold'] = -1

kf = model_selection.StratifiedKFold(n_splits = 5)

for fold_ ,(t_,v_) in enumerate(kf.split(X= df, y=y)):
    df.loc[v_, 'kfold'] = fold_
    
for f in range(5):
    df_train = df[df.kfold != f].reset_index(drop=True)
    df_test = df[df.kfold == f].reset_index(drop=True)
    
    count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern=None)
    
    x_train = count_vec.fit_transform(df_train.Sentence)
    x_test = count_vec.transform(df_test.Sentence)
    
    model = MultinomialNB(alpha = 1.6)
    
    model.fit(x_train, df_train.Sentiment)
    
    y_pred = model.predict(x_test)
    
    accuracy_precision = precision_score(df_test.Sentiment, y_pred, average = 'micro')
    
    #accuracy_recall = recall_score(df_test.Sentiment, y_pred)
    
    print('precision score:', accuracy_precision)
    #print('recall score:', accuracy_recall)
    print("")
    

print(classification_report(df_test.Sentiment, y_pred, labels=[1,-1,0]))    

precision score: 0.7262617621899059

precision score: 0.7117194183062446

precision score: 0.7148972602739726

precision score: 0.711472602739726

precision score: 0.699486301369863

              precision    recall  f1-score   support

           1       0.73      0.61      0.67       370
          -1       0.45      0.19      0.27       172
           0       0.71      0.89      0.79       626

    accuracy                           0.70      1168
   macro avg       0.63      0.57      0.58      1168
weighted avg       0.68      0.70      0.67      1168



In [24]:
df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df['kfold'] = -1

kf = model_selection.StratifiedKFold(n_splits = 5)

for fold_ ,(t_,v_) in enumerate(kf.split(X= df, y=y)):
    df.loc[v_, 'kfold'] = fold_
    
for f in range(5):
    df_train = df[df.kfold != f].reset_index(drop=True)
    df_test = df[df.kfold == f].reset_index(drop=True)
    
    count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern=None)
    
    x_train = count_vec.fit_transform(df_train.Sentence)
    x_test = count_vec.transform(df_test.Sentence)
    
    model = SVC(C= 100, gamma =  0.001, kernel = 'rbf')
    
    model.fit(x_train, df_train.Sentiment)
    
    y_pred = model.predict(x_test)
    
    accuracy_precision = precision_score(df_test.Sentiment, y_pred, average = 'micro')
    
    #accuracy_recall = recall_score(df_test.Sentiment, y_pred)
    
    print('precision score:', accuracy_precision)
    #print('recall score:', accuracy_recall)
    print("")
    

print(classification_report(df_test.Sentiment, y_pred, labels=[1,-1,0]))   

precision score: 0.6826347305389222

precision score: 0.7014542343883661

precision score: 0.6892123287671232

precision score: 0.6952054794520548

precision score: 0.6866438356164384

              precision    recall  f1-score   support

           1       0.77      0.66      0.71       370
          -1       0.29      0.20      0.24       172
           0       0.72      0.83      0.77       626

    accuracy                           0.69      1168
   macro avg       0.59      0.57      0.57      1168
weighted avg       0.67      0.69      0.67      1168



**The model has performed worser than the previous above model, we can also create different pipelines of different machine learning models and compare them with both BOW and tfidf models**

**More models including deep learning models are yet to come**


# Model- 8 LSTM using glove word2vec

In [25]:
# loading pretrained google news word2vec embedding 300D
from gensim.models import KeyedVectors
word2vec_pretrained = KeyedVectors.load_word2vec_format("../input/googles-trained-word2vec-model-in-python/GoogleNews-vectors-negative300.bin",binary=True)
word2vec_pretrained_dict = dict(zip(word2vec_pretrained.key_to_index.keys(), 
                                    word2vec_pretrained.vectors))

In [26]:
df['Sentence'] = df['Sentence'].apply(process_text)

X_train,X_test,y_train,y_test = train_test_split(df.Sentence, df.Sentiment, test_size = 0.2, random_state = 42)

In [27]:
y_train_enc = np_utils.to_categorical(y_train, 3)
y_test_enc = np_utils.to_categorical(y_test, 3)

In [28]:
token = tf.keras.preprocessing.text.Tokenizer(num_words=None)

token.fit_on_texts(list(X_train) + list(X_test))
xtrain_seq = token.texts_to_sequences(X_train)
xtest_seq = token.texts_to_sequences(X_test)

#zero pad sequences 

xtrain_pad = sequence.pad_sequences(xtrain_seq)
xtest_pad = sequence.pad_sequences(xtest_seq)

word_index = token.word_index

In [29]:
#create embedding matrix for words that we have in dataset

embedding_matrix = np.zeros((len(word_index)+1, 300))
for word,i in word_index.items():
    embedding_vector = word2vec_pretrained_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        

In [30]:
# A simple LSTM with two dense layers

model = Sequential()
model.add(Embedding(len(word_index)+1,300,weights=[embedding_matrix], trainable = False))

model.add(SpatialDropout1D(0.3))
model.add(LSTM(300, dropout = 0.3, recurrent_dropout = 0.3))

model.add(Dense(1024 , activation = 'relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = [tf.keras.metrics.Precision()])

2022-03-11 04:10:50.220617: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-03-11 04:10:50.231497: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [31]:
model.fit(xtrain_pad, y=y_train_enc, batch_size = 512, epochs =100, verbose=1, validation_data = (xtest_pad, y_test_enc))

Epoch 1/100


2022-03-11 04:10:54.470798: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 30720000 exceeds 10% of free system memory.
2022-03-11 04:10:54.497224: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 30720000 exceeds 10% of free system memory.
2022-03-11 04:10:56.104002: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 30720000 exceeds 10% of free system memory.
2022-03-11 04:10:56.201103: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 30720000 exceeds 10% of free system memory.


 1/10 [==>...........................] - ETA: 58s - loss: 1.1072 - precision: 0.0000e+00

2022-03-11 04:10:57.676219: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 30720000 exceeds 10% of free system memory.


10/10 [==============================] - 27s 2s/step - loss: 1.0037 - precision: 0.5660 - val_loss: 0.9320 - val_precision: 0.7268
Epoch 2/100
10/10 [==============================] - 23s 2s/step - loss: 0.9198 - precision: 0.6690 - val_loss: 0.9513 - val_precision: 0.6989
Epoch 3/100
10/10 [==============================] - 22s 2s/step - loss: 0.8866 - precision: 0.7359 - val_loss: 0.9019 - val_precision: 0.6613
Epoch 4/100
10/10 [==============================] - 22s 2s/step - loss: 0.8712 - precision: 0.7196 - val_loss: 0.8393 - val_precision: 0.7084
Epoch 5/100
10/10 [==============================] - 23s 2s/step - loss: 0.8487 - precision: 0.7258 - val_loss: 0.8016 - val_precision: 0.7295
Epoch 6/100
10/10 [==============================] - 22s 2s/step - loss: 0.8044 - precision: 0.7233 - val_loss: 0.7638 - val_precision: 0.7433
Epoch 7/100
10/10 [==============================] - 22s 2s/step - loss: 0.7811 - precision: 0.7299 - val_loss: 0.7607 - val_precision: 0.7686
Epoch 8/100

**A problem with training neural networks is in the choice of the number of training epochs to use.Too many epochs can lead to overfitting of the training dataset, whereas too few may result in an underfit model. Early stopping is a method that allows you to specify an arbitrary large number of training epochs and stop training once the model performance stops improving on a hold out validation dataset.**

**So lets use earlystopping, I see that after 43 epoch out validation loss has increased so lets use early stopping to fix that**

# Model 9 LSTM word2vec + early stopping

In [32]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = [tf.keras.metrics.Precision()])

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=y_train_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data = (xtest_pad, y_test_enc), callbacks=[earlystop])

Epoch 1/100
10/10 [==============================] - 25s 2s/step - loss: 1.0016 - precision_1: 0.5536 - val_loss: 0.9593 - val_precision_1: 0.7619
Epoch 2/100
10/10 [==============================] - 23s 2s/step - loss: 0.9347 - precision_1: 0.6705 - val_loss: 0.8811 - val_precision_1: 0.7448
Epoch 3/100
10/10 [==============================] - 22s 2s/step - loss: 0.8756 - precision_1: 0.7177 - val_loss: 0.8544 - val_precision_1: 0.7001
Epoch 4/100
10/10 [==============================] - 22s 2s/step - loss: 0.8515 - precision_1: 0.7306 - val_loss: 0.8108 - val_precision_1: 0.7809
Epoch 5/100
10/10 [==============================] - 23s 2s/step - loss: 0.8117 - precision_1: 0.7441 - val_loss: 0.8801 - val_precision_1: 0.6866
Epoch 6/100
10/10 [==============================] - 22s 2s/step - loss: 0.8111 - precision_1: 0.7318 - val_loss: 0.7424 - val_precision_1: 0.7670
Epoch 7/100
10/10 [==============================] - 22s 2s/step - loss: 0.7652 - precision_1: 0.7387 - val_loss: 0.71

**Precision is close to 80 now, so there is an improvment in precision score. Lets see the classification report**

In [33]:
y_pred = model.predict(xtest_pad)

# Here '2' is '-1' in previous reports

print(classification_report(np.argmax(y_test_enc,axis=1), np.argmax(y_pred,axis=1), labels=[0,1,2]))   

              precision    recall  f1-score   support

           0       0.75      0.81      0.78       620
           1       0.84      0.68      0.75       381
           2       0.48      0.54      0.51       168

    accuracy                           0.73      1169
   macro avg       0.69      0.68      0.68      1169
weighted avg       0.74      0.73      0.73      1169



**Its evident that there is a improvement in all the clasification classes, hence this will be our new baseline model.** 

# Model 10 bidirectional LSTM 

In [34]:
model_bi = Sequential()
model_bi.add(Embedding(len(word_index)+1, 300, weights=[embedding_matrix], trainable = False))

model_bi.add(SpatialDropout1D(0.3))
model_bi.add(Bidirectional(LSTM(300, dropout = 0.3, recurrent_dropout = 0.3)))

model_bi.add(Dense(1024, activation = 'relu'))
model_bi.add(Dropout(0.8))

model_bi.add(Dense(1024, activation = 'relu'))
model_bi.add(Dropout(0.8))

model_bi.add(Dense(3))
model_bi.add(Activation('softmax'))
model_bi.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = tf.keras.metrics.Precision())

earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3, verbose = 0, mode = 'auto')
model_bi.fit(xtrain_pad, y=y_train_enc, batch_size = 512, epochs = 100, verbose=1, validation_data = (xtest_pad, y_test_enc),callbacks = [earlystop])

y_pred = model_bi.predict(xtest_pad)



Epoch 1/100
10/10 [==============================] - 51s 5s/step - loss: 0.9972 - precision_2: 0.5596 - val_loss: 0.9385 - val_precision_2: 0.7343
Epoch 2/100
10/10 [==============================] - 43s 4s/step - loss: 0.9178 - precision_2: 0.6558 - val_loss: 0.8764 - val_precision_2: 0.7553
Epoch 3/100
10/10 [==============================] - 44s 4s/step - loss: 0.8695 - precision_2: 0.7328 - val_loss: 0.8563 - val_precision_2: 0.8256
Epoch 4/100
10/10 [==============================] - 45s 4s/step - loss: 0.8435 - precision_2: 0.7347 - val_loss: 0.8081 - val_precision_2: 0.7496
Epoch 5/100
10/10 [==============================] - 43s 4s/step - loss: 0.8205 - precision_2: 0.7220 - val_loss: 0.7833 - val_precision_2: 0.7627
Epoch 6/100
10/10 [==============================] - 44s 4s/step - loss: 0.7942 - precision_2: 0.7298 - val_loss: 0.7921 - val_precision_2: 0.7986
Epoch 7/100
10/10 [==============================] - 45s 5s/step - loss: 0.7789 - precision_2: 0.7394 - val_loss: 0.72

In [35]:
print(classification_report(np.argmax(y_test_enc, axis=1), np.argmax(y_pred , axis=1), labels = [0,1,2]))

              precision    recall  f1-score   support

           0       0.74      0.90      0.81       620
           1       0.81      0.70      0.75       381
           2       0.56      0.30      0.39       168

    accuracy                           0.75      1169
   macro avg       0.70      0.63      0.65      1169
weighted avg       0.74      0.75      0.73      1169



**Well bidrectional LSTM did not beat our baseline LSTM model so we are not going to consider this model**

# Model 11 GRU 2 layers

In [36]:
model_gru = Sequential()
model_gru.add(Embedding(len(word_index)+1, 300, weights = [embedding_matrix], trainable=False))

model_gru.add(SpatialDropout1D(0.3))
model_gru.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences = True))
model_gru.add(GRU(300, dropout = 0.3, recurrent_dropout = 0.3))

model_gru.add(Dense(1024, activation = 'relu'))
model_gru.add(Dropout(0.8))

model_gru.add(Dense(1024, activation = 'relu'))
model_gru.add(Dropout(0.8))

model_gru.add(Dense(3, activation = 'softmax'))
model_gru.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = tf.keras.metrics.Precision())

earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3, verbose = 0, mode= 'auto')

model_gru.fit(xtrain_pad, y=y_train_enc, batch_size = 512, epochs = 100, verbose=1, validation_data = (xtest_pad, y_test_enc),callbacks = [earlystop])

y_pred = model_gru.predict(xtest_pad)

print(classification_report(np.argmax(y_test_enc, axis=1), np.argmax(y_pred , axis=1), labels = [0,1,2]))

Epoch 1/100
10/10 [==============================] - 41s 4s/step - loss: 1.0093 - precision_3: 0.5587 - val_loss: 0.9452 - val_precision_3: 0.7292
Epoch 2/100
10/10 [==============================] - 37s 4s/step - loss: 0.9217 - precision_3: 0.6558 - val_loss: 0.8765 - val_precision_3: 0.7454
Epoch 3/100
10/10 [==============================] - 36s 4s/step - loss: 0.8808 - precision_3: 0.7214 - val_loss: 0.8542 - val_precision_3: 0.7607
Epoch 4/100
10/10 [==============================] - 37s 4s/step - loss: 0.8646 - precision_3: 0.7121 - val_loss: 0.8230 - val_precision_3: 0.7401
Epoch 5/100
10/10 [==============================] - 38s 4s/step - loss: 0.8346 - precision_3: 0.7054 - val_loss: 0.7885 - val_precision_3: 0.7903
Epoch 6/100
10/10 [==============================] - 37s 4s/step - loss: 0.8001 - precision_3: 0.7233 - val_loss: 0.7386 - val_precision_3: 0.7672
Epoch 7/100
10/10 [==============================] - 38s 4s/step - loss: 0.7966 - precision_3: 0.7212 - val_loss: 0.81

**Well not much difference between baseline single layer LSTM model to GRU model, but it looks our baseline model is still better.**

**Lets add one more LSTM layer to baseline model and see what happens**

# Model 12 LSTM multiple layers

In [37]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300, dropout=0.3, recurrent_dropout=0.3 ,  return_sequences = True))
model.add(LSTM(300, dropout = 0.3, recurrent_dropout = 0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = [tf.keras.metrics.Precision()])

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=y_train_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data = (xtest_pad, y_test_enc), callbacks=[earlystop])

y_pred = model.predict(xtest_pad)
print(classification_report(np.argmax(y_test_enc, axis=1), np.argmax(y_pred, axis=1), labels=[0,1,2]))

Epoch 1/100
10/10 [==============================] - 53s 5s/step - loss: 1.0124 - precision_4: 0.5763 - val_loss: 0.9283 - val_precision_4: 0.6295
Epoch 2/100
10/10 [==============================] - 46s 5s/step - loss: 0.9200 - precision_4: 0.6727 - val_loss: 0.8817 - val_precision_4: 0.6846
Epoch 3/100
10/10 [==============================] - 46s 5s/step - loss: 0.8736 - precision_4: 0.7307 - val_loss: 0.8506 - val_precision_4: 0.8113
Epoch 4/100
10/10 [==============================] - 47s 5s/step - loss: 0.8362 - precision_4: 0.7200 - val_loss: 0.7787 - val_precision_4: 0.7817
Epoch 5/100
10/10 [==============================] - 46s 5s/step - loss: 0.7802 - precision_4: 0.7487 - val_loss: 0.8040 - val_precision_4: 0.7574
Epoch 6/100
10/10 [==============================] - 46s 5s/step - loss: 0.7960 - precision_4: 0.7314 - val_loss: 0.7376 - val_precision_4: 0.7785
Epoch 7/100
10/10 [==============================] - 47s 5s/step - loss: 0.7722 - precision_4: 0.7341 - val_loss: 0.70

**Well the additional LSTM layer has imporoved the first 2 labels and did not imporove much for third label**

**I will stop this notebook here, it has become very big already, maybe i will create an other notebook only for transfer learning! anyway there are many ways to model in machine learning, it depends on your dataset which model suits the best.** 

**Until then keep hustling! See you next time**